In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf

import random, glob, os
import numpy as np

from pydub import AudioSegment
from pydub import effects
from utils.refactored_common import *
# from utils.refactored_common import unision_shuffled_copies
from tqdm.notebook import tqdm

try :
    from keras.utils import Sequence #   sequence =  keras.utils.Sequence
except:
    from keras.utils.all_utils import Sequence


# import tensorflow_io as tfio

import soundfile as sf
import audioflux
from scipy import signal

In [82]:
cfg = '''
feature:
  raw_mode: 0    # return raw data instead of nmels
  n_mels: 128    # 1, 128 typically
  frames : -1 #36 #128 #-1 #128   # 64000, 128 typically
  n_fft: 2048
  hop_length: 240 #160 #512 #1200  # 160 # [ 240 -> for impulse ]
  win_length: 2048 # or 2048 -> [for impulse ]
  #power: 2.0
  steps: 1 #24 #128
  flatten: False
  incNG: 0 #0.05
  return_label: False
  normalize: False
  Fs : 48000
  duration : 200 # for impulse mel -> 120 #ms for impulse processor.
  includePre : False
  #frames : -1
  #steps : 1
  ignLimit : 128
  threshold : 2
  sampling_rate : 4000
  cutoff_freq : 100
  window : 'hann'
  window_types : # for audioflux winodw types
    hann : 1
    hamm : 2
    blackman : 3
    kaiser : 4
    gauss : 8
    tukey : 13
  
'''

In [2]:
params  = yaml_load("cfg.yaml")

In [5]:
from generators import base_generator_audio as BASE
from  curricula import selection
from models import base_cnn, transformer_classifier, wavenet

In [ ]:
#AutoEncoderWavenetBase(height, width, kernel_size=kernel_size, hidden_channels=hidden_channels, beta=0)
        # if loss == 'customLoss' :

In [8]:
run_name = "audio_mnist__CNN__CNN__rho_selection"   #! Convention: "dataset__irred_model__target_model__curriculum"
irred_chkpt = f"results/{run_name}/checkpoints/mnist_spec_CNN_small.keras"
control_chkpt = f"results/{run_name}/checkpoints/mnist_spec_CNN_large.keras"
target_chkpt = f"results/{run_name}/checkpoints/mnist_spec_CNN_large_rho.keras"

irred_log = f"results/{run_name}/logs/mnist_spec_CNN_small"
control_log = f"results/{run_name}/logs/mnist_spec_CNN_large"
target_log = f"results/{run_name}/logs/mnist_spec_CNN_large_rho"

width = 25
height = 128
num_classes = 10
epochs = 10
cfg = "cfg.yaml"
base_dir = "data/audio/mnist/"
minibatch_size = 0.5
batch_size = 32
ext = 'wav'
return_spec = True
return_fft = False



params = yaml_load(cfg)
run_params = {
    "run_name": run_name,
    "irred_chkpt": irred_chkpt,
    "control_chkpt": control_chkpt,
    "target_chkpt": target_chkpt,
    "irred_log": irred_log,
    "control_log": control_log,
    "target_log": target_log,
    "width": width,
    "height": height,
    "num_classes": num_classes,
    "epochs": epochs,
    "cfg": cfg,
    "base_dir": base_dir,
    "minibatch_size": minibatch_size,
    "batch_size": batch_size,
    "ext": ext,
    "return_spec": return_spec,
}



In [9]:
#! Basic dataloaders
train_gen = BASE.BaseClassificationGenerator(params, base_dir, batch_size, gentype='train', return_spec=return_spec, return_fft=return_fft, ext=ext)

holdout_gen = BASE.BaseClassificationGenerator(params, base_dir, batch_size, gentype='val', return_spec=return_spec, return_fft=return_fft, ext=ext)

test_gen = BASE.BaseClassificationGenerator(params, base_dir, batch_size, gentype='test', return_spec=return_spec, return_fft=return_fft, ext=ext)

base_class_params = train_gen.toJSON()

run_params["base_dataloader_params"] = base_class_params



In [10]:
a, b = train_gen.__getitem__(0)
print(a.shape, b.shape)

(32, 25, 128) (32, 10)


In [11]:
model = wavenet.AutoEncoderWavenetBase(
    height, width, kernel_size=4, hidden_channels = [128, 96, 64, 48, 32, 24, 16, 8, 4, 2]
)

In [17]:
#! Put a dense layer on top of the encoder to get the classification

input_ = tf.keras.layers.Input(shape=(width, height))
x = model(input_)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

trainey_model = tf.keras.Model(inputs=input_, outputs=x)

In [18]:

trainey_model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy']
)
# model.summary()

In [20]:
trainey_model.fit(train_gen, epochs=epochs, validation_data=test_gen)

Epoch 1/10


/home/juggernautjha/.conda/envs/ravijas/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


468/468 ━━━━━━━━━━━━━━━━━━━━ 141s 277ms/step - accuracy: 0.1808 - loss: 2.2306 - val_accuracy: 0.1032 - val_loss: 2.7531
Epoch 2/10
468/468 ━━━━━━━━━━━━━━━━━━━━ 138s 295ms/step - accuracy: 0.5616 - loss: 1.3313 - val_accuracy: 0.0413 - val_loss: 3.4541
Epoch 3/10
164/468 ━━━━━━━━━━━━━━━━━━━━ 1:07 222ms/step - accuracy: 0.7679 - loss: 0.7401

KeyboardInterrupt: 